In [1]:
import pandas as pd
import json
from openpyxl import load_workbook
import re

In [2]:
xls = pd.ExcelFile("MAPPA - Cerchio Editoriale Bari.xlsx")
dfs = {}
for sheet in xls.sheet_names:
    dfs[sheet] = pd.read_excel(xls, sheet)

In [3]:
def get_hyperlinks(sheet_name):
    ws = wb[sheet_name]
    hyperlinks = {}
    
    for row_idx, row in enumerate(ws.iter_rows(min_row=2), start=0):  # Skip header row
        for col_idx, cell in enumerate(row):
            if cell.hyperlink:
                if row_idx not in hyperlinks:
                    hyperlinks[row_idx] = {}
                # Get column name from header
                col_name = ws.cell(1, col_idx + 1).value
                hyperlinks[row_idx][col_name] = {
                    'text': cell.value,
                    'url': cell.hyperlink.target
                }
    return hyperlinks

wb = load_workbook("MAPPA - Cerchio Editoriale Bari.xlsx")

In [4]:
import re

def parse_orari_apertura(orari_string):
    """
    Parse the opening hours string and return structured data.
    Returns a dict with days as keys and opening/closing times as values.
    """
    default_hours = {
        "lunedì": {"apertura": [], "chiusura": []},
        "martedì": {"apertura": [], "chiusura": []},
        "mercoledì": {"apertura": [], "chiusura": []},
        "giovedì": {"apertura": [], "chiusura": []},
        "venerdì": {"apertura": [], "chiusura": []},
        "sabato": {"apertura": [], "chiusura": []},
        "domenica": {"apertura": [], "chiusura": []}
    }
    
    # If empty or not a string, return default
    if not orari_string or not isinstance(orari_string, str):
        return default_hours
    
    result = {}
    lines = orari_string.strip().split('\n')
    
    if orari_string.lower() == 'sempre aperto':
        for day in default_hours:
            result[day] = {"apertura": [0], "chiusura": [24]}
    elif 'orari variabili' in orari_string:
        for day in default_hours:
            result[day] = {"apertura": [], "chiusura": []}
    else:
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Split by colon to get day and hours
            if ':' not in line:
                continue
                
            parts = line.split(':', 1)
            day = parts[0].strip().lower()
            hours_str = parts[1].strip().lower()
            
            # Check if closed
            if 'chiuso' in hours_str or 'chiusa' in hours_str:
                result[day] = {"apertura": [], "chiusura": []}
            else:
                # Parse time ranges
                apertura = []
                chiusura = []
                
                # Split by comma for multiple time ranges
                time_ranges = hours_str.split(',')
                
                for time_range in time_ranges:
                    time_range = time_range.strip()
                    # Match pattern like "11:30-13" or "11-13" or "0:30-24"
                    match = re.search(r'(\d+(?::\d+)?)\s*-\s*(\d+(?::\d+)?)', time_range)
                    if match:
                        apertura.append(match.group(1))
                        chiusura.append(match.group(2))
                
                result[day] = {"apertura": apertura, "chiusura": chiusura}
    
    return result

# Test
result = parse_orari_apertura("""lunedì: 0:30-24
                        martedì: 11:30-13, 14-20
                        mercoledì: 11-13, 14-20
                        giovedì: 11-13, 14-20
                        venerdì: 11-13, 14-20
                        sabato: 11-13, 14-20
                        domenica: chiuso""")
result

{'lunedì': {'apertura': ['0:30'], 'chiusura': ['24']},
 'martedì': {'apertura': ['11:30', '14'], 'chiusura': ['13', '20']},
 'mercoledì': {'apertura': ['11', '14'], 'chiusura': ['13', '20']},
 'giovedì': {'apertura': ['11', '14'], 'chiusura': ['13', '20']},
 'venerdì': {'apertura': ['11', '14'], 'chiusura': ['13', '20']},
 'sabato': {'apertura': ['11', '14'], 'chiusura': ['13', '20']},
 'domenica': {'apertura': [], 'chiusura': []}}

In [5]:
allKeys = []
for sheet in dfs:
    allKeys.extend(list(dfs[sheet].keys()))
    allKeys.append("Categoria")
allKeys = set(allKeys)

In [6]:
dfs.keys()

dict_keys(['Spazi di lotta', 'Spazi di studio', 'Spazi di aggregazione', 'Spazi vuoti e svuotati', 'Spazi del gioco'])

In [7]:
allKeys

{'Capienza',
 'Categoria',
 'Descrizione attività',
 'Funzionalità',
 'Indirizzo',
 'Indirizzo google',
 'Latitudine',
 'Link',
 'Link social',
 'Longitudine',
 'Note',
 'Orari e giorni di apertura ',
 'Spazio'}

In [8]:
final = []
for sheet in dfs:
    df = dfs[sheet]
    hyperlinks = get_hyperlinks(sheet)
    df = df.where(pd.notna(df), None)

    for idx, spazio in enumerate(df.to_dict(orient='index').values()):
        spazio["Categoria"] = sheet
        
        if idx in hyperlinks:
            for col_name, link_info in hyperlinks[idx].items():
                spazio[col_name] = link_info['text']
                spazio[f"{col_name}"] = link_info['url']
        
        spazio = {key: ("" if pd.isna(value) or value is None else value) for key, value in spazio.items()}
            
        if "Orari e giorni di apertura " in spazio:
            orari_string = spazio.get("Orari e giorni di apertura ", "")
            print(orari_string)
            spazio["orari apertura"] = parse_orari_apertura(orari_string)
            print (spazio["orari apertura"])
            print ("____________________")
            del spazio["Orari e giorni di apertura "]

        final.append(spazio)

lunedì: 7:30 - 22:30
martedì: 7:30 - 22:30
mercoledì: 7:30 - 22:30
giovedì: 7:30 - 22:30
venerdì: 7:30 - 22:30
sabato: 8:00 - 19:30
domenica: 9:00 - 16:30
{'lunedì': {'apertura': ['7:30'], 'chiusura': ['22:30']}, 'martedì': {'apertura': ['7:30'], 'chiusura': ['22:30']}, 'mercoledì': {'apertura': ['7:30'], 'chiusura': ['22:30']}, 'giovedì': {'apertura': ['7:30'], 'chiusura': ['22:30']}, 'venerdì': {'apertura': ['7:30'], 'chiusura': ['22:30']}, 'sabato': {'apertura': ['8:00'], 'chiusura': ['19:30']}, 'domenica': {'apertura': ['9:00'], 'chiusura': ['16:30']}}
____________________
lunedì: 8:30 - 19:30
martedì: 8:30 - 19:30
mercoledì: 8:30 - 19:30
giovedì: 8:30 - 19:30
venerdì: 8:30 - 19:30
sabato: 8:30 - 13:30
domenica: chiuso
{'lunedì': {'apertura': ['8:30'], 'chiusura': ['19:30']}, 'martedì': {'apertura': ['8:30'], 'chiusura': ['19:30']}, 'mercoledì': {'apertura': ['8:30'], 'chiusura': ['19:30']}, 'giovedì': {'apertura': ['8:30'], 'chiusura': ['19:30']}, 'venerdì': {'apertura': ['8:30'],

In [9]:
# with open ("Spazi.json", "w", encoding='UTF-8') as f:
#     json.dump(final, f, indent=4, ensure_ascii=False)

In [10]:
categories = set()
for spazio in final:
    categories.add(spazio["Categoria"])
categories

{'Spazi del gioco', 'Spazi di lotta', 'Spazi di studio'}

In [11]:
spaziDiAggregazione = []
spaziDiLotta = []
spaziDiStudio = []
for spazio in final:
    if spazio["Categoria"] == 'Spazi del gioco' or spazio["Categoria"] == 'Spazi di aggregazione':
        spaziDiAggregazione.append(spazio)
    elif spazio["Categoria"] == 'Spazi di lotta':
        spaziDiLotta.append(spazio)
    elif spazio["Categoria"] == 'Spazi di studio':
        spaziDiStudio.append(spazio)

In [12]:
with open ("questionario.json", encoding="utf-8") as f:
    questionarioData = json.load(f)

In [13]:
questionarioData[0].keys(), spaziDiAggregazione[0].keys(), spaziDiAggregazione[0]["Categoria"]

(dict_keys(['Latitudine', 'Longitudine', 'name', 'value']),
 dict_keys(['Spazio', 'Categoria', 'Indirizzo', 'Funzionalità', 'Capienza', 'Link', 'Note', 'Latitudine', 'Longitudine', 'orari apertura']),
 'Spazi del gioco')

In [14]:
for elem in questionarioData:
    spaziDiAggregazione.append({
        "Spazio":elem["name"],
        "Categoria":'Spazi del gioco',
        'Latitudine': elem['Latitudine'], 
        'Longitudine': elem['Longitudine']
    })

In [15]:
spaziDiAggregazione

[{'Spazio': 'Libertà: giardini Piazza Umberto',
  'Categoria': 'Spazi del gioco',
  'Indirizzo': 'Giardini di Piazza Umberto I, Piazza Umberto I, 27, 70121 Bari BA',
  'Funzionalità': 'Area giochi bambini, passeggio, luogo di passaggio poichè vicino stazione e uni',
  'Capienza': '/',
  'Link': 'https://maps.app.goo.gl/oQZt92jWadbGoJvW6',
  'Note': '',
  'Latitudine': 41.120762765048255,
  'Longitudine': 16.86995201385369,
  'orari apertura': {'lunedì': {'apertura': [0], 'chiusura': [24]},
   'martedì': {'apertura': [0], 'chiusura': [24]},
   'mercoledì': {'apertura': [0], 'chiusura': [24]},
   'giovedì': {'apertura': [0], 'chiusura': [24]},
   'venerdì': {'apertura': [0], 'chiusura': [24]},
   'sabato': {'apertura': [0], 'chiusura': [24]},
   'domenica': {'apertura': [0], 'chiusura': [24]}}},
 {'Spazio': 'Madonella: Giardini Baden Powell',
  'Categoria': 'Spazi del gioco',
  'Indirizzo': 'Giardini Baden Powell, Piazza Antonio Gramsci, 3, 70126 Bari BA',
  'Funzionalità': 'Area giochi 

In [16]:
with open (r"SpaziDiAggregazione\resources\SpaziDiAggregazione.json", "w", encoding='utf-8') as f:
    json.dump(spaziDiAggregazione, f, indent=4, ensure_ascii=False)
with open (r"SpaziDiLotta\resources\SpaziDiLotta.json", "w", encoding='utf-8') as f:
    json.dump(spaziDiLotta, f, indent=4, ensure_ascii=False)
with open (r"SpaziDiStudio\resources\SpaziDiStudio.json", "w", encoding='utf-8') as f:
    json.dump(spaziDiStudio, f, indent=4, ensure_ascii=False)
    